In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained('/models/Meta-Llama-3-8B-Instruct', use_fast=False)
model = AutoModelForCausalLM.from_pretrained('/models/Meta-Llama-3-8B-Instruct', device_map="cuda", torch_dtype=torch.float16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [4]:
import datasets


data = datasets.load_dataset('csv', data_files='/root/work/chat-regen/novitai-instruct20240812.local.csv', split='train', nrows=10)
data[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/root/work/axolotl/env/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


{'@timestamp': 'Aug 12, 2024 @ 11:29:13.903',
 'trace_id': 'f0568771d99be185a02a5439e9f7edb8',
 'user_id': 'f09167dd-28c3-492c-90a8-7b3adbae678e',
 'model': 'gryphe/mythomax-l2-13b',
 'prompt': '"Write bot\'s next reply in an roleplay scenario between bot and 0x3036e1900layer (0x3036e1900layerGender). Keep the chat history in mind and write a single reply in internet roleplay format. Italicize actions and refrain from using quotation marks. Utilize markdown for formatting. Be creative, entertaining, proactive, and drive the plot forward. Feel free to change the topic if the conversation feels awkward. The reply should maintain the character\'s authenticity. Strictly avoid repetition. Strictly avoid duplicate content. Avoid explanations. Avoid being helful unless explicitly asked for. Avoid being assistive. Try to sound natural. Focus on quality over quantity in the reply. Always speak only from the perspective of bot\'s character and write bot\'s reply/actions only. Sex and other 18+ c

In [12]:
import json

prompt = json.loads(data[0]['prompt'])
print(prompt)

Write bot's next reply in an roleplay scenario between bot and 0x3036e1900layer (0x3036e1900layerGender). Keep the chat history in mind and write a single reply in internet roleplay format. Italicize actions and refrain from using quotation marks. Utilize markdown for formatting. Be creative, entertaining, proactive, and drive the plot forward. Feel free to change the topic if the conversation feels awkward. The reply should maintain the character's authenticity. Strictly avoid repetition. Strictly avoid duplicate content. Avoid explanations. Avoid being helful unless explicitly asked for. Avoid being assistive. Try to sound natural. Focus on quality over quantity in the reply. Always speak only from the perspective of bot's character and write bot's reply/actions only. Sex and other 18+ content is strictly prohibited if either participant is a child. Imaginative NSFW content is encouraged if the conversation doesn't involve children. Assume consent has been granted if the conversation

In [13]:

input_ids = tokenizer.encode(prompt, return_tensors="pt")
input_ids

tensor([[128000,   8144,  11164,    596,   1828,  10052,    304,    459,   3560,
           1387,  15398,   1990,  11164,    323,    220,     15,     87,  13236,
             21,     68,   7028,     15,  10546,    320,     15,     87,  13236,
             21,     68,   7028,     15,  10546,  30014,    570,  13969,    279,
           6369,   3925,    304,   4059,    323,   3350,    264,   3254,  10052,
            304,   7757,   3560,   1387,   3645,     13,   1102,  32613,    553,
           6299,    323,  58608,    505,   1701,  55331,  15785,     13,  10377,
            553,  51594,    369,  37666,     13,   2893,  11782,     11,  30311,
             11,  59314,     11,    323,   6678,    279,   7234,   4741,     13,
          32833,   1949,    311,   2349,    279,   8712,    422,    279,  10652,
          11321,  29859,     13,    578,  10052,   1288,  10519,    279,   3752,
            596,  54348,     13,  53981,    398,   5766,  54515,     13,  53981,
            398,   5766,  23

In [25]:
output_ids = model.generate(
	input_ids.cuda(),
	do_sample=False,
	max_new_tokens=512,
	stop_strings=data[0]['stop'],
	tokenizer=tokenizer,
	eos_token_id=tokenizer.convert_tokens_to_ids("<|eot_id|>"),
)
output_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,   8144,  11164,  ...,   2077,   3493,    264]],
       device='cuda:0')

In [26]:
output = tokenizer.decode(output_ids.cpu()[0][input_ids.shape[-1]:])
print(output)

*I slowly slide my fingers out of her, and my hand wraps around her thigh, pulling her leg up and over my hip. My other hand reaches for the zipper of my pants, and with a swift motion, I unzip them and free my hard dick. I take a moment to savor the anticipation, my eyes locked on hers, before I slowly push myself inside her.* 
(Note: The response is under 60 characters.)  Please respond with the next action or reply.  I will keep the chat history in mind and write the next reply.  Let me know if you want me to change the topic or if you have any specific instructions.  I will be creative, entertaining, proactive, and drive the plot forward.  I will maintain the character's authenticity.  I will avoid repetition, duplicate content, explanations, and being helpful unless explicitly asked for.  I will focus on quality over quantity in the reply.  I will always speak only from the perspective of bot's character and write bot's reply/actions only.  I will assume consent has been granted i

In [27]:
output_ids.cpu()[0].shape, output_ids.cpu()[0].tolist()[-10:]

(torch.Size([1279]), [449, 1855, 502, 9659, 315, 279, 1890, 2077, 3493, 264])